In [ ]:
!pip install opencv-python
from os import listdir
import PIL as pil
import cv2
from datetime import datetime
import pandas as pd
import sgolay2 as sgf2
from scipy import signal
import numpy as np
from skimage import exposure
import matplotlib.pyplot as plt

In [ ]:
try:
    df = pd.read_csv('data.csv')
except: None

In [ ]:
if df.empty:
    files = [file for file in listdir('images/') if file.endswith('tiff')]
else:
    files = [file for file in listdir('images/') if file.endswith('tiff') and file not in df[filename]]

In [ ]:
def get_images(filess):
    return [cv2.imread('images/'+file, cv2.IMREAD_ANYDEPTH | cv2.IMREAD_GRAYSCALE) for file in filess]
def get_image(file):
    return cv2.imread('images/'+file, cv2.IMREAD_ANYDEPTH | cv2.IMREAD_GRAYSCALE)

In [ ]:
def filter_image(image):
    #return image
    return sgf2.SGolayFilter2(window_size=9, poly_order=3)(image)

In [ ]:
def contrast_image(image):
    return exposure.rescale_intensity(image, in_range=tuple(np.percentile(image, (10.0, 95.0))))

In [ ]:
def convolve_image(image):
    return signal.convolve(image, [[-1.], [-1.], [-1.],
                                    [-1.], [8.], [-1.],
                                    [-1.], [-1.], [-1.]])

In [180]:
def save_data(data):
    try:
        pd.read_csv('data.csv')
        header = False
    except:
        header = True
    data.to_csv('data.csv', mode='a', header=header, index=False)

In [239]:
def get_files():
    try:
        df = pd.read_csv('data.csv')
    except: df = pd.DataFrame(columns=[1])
    print(df)
    if df.empty:
        return [file for file in listdir('images/') if file.endswith('tiff')]
    else:
        return [file for file in listdir('images/') if file.endswith('tiff') and not(df['filename'].str.contains(file).any())]

In [240]:
def run():
    files = get_files()
    for i in files:
        img = get_image(i)
        time = datetime.now()
        con_img = convolve_image(contrast_image(img))
        filter_img = filter_image(con_img)
        sur = abs((filter_img))>np.max(filter_img)*0.2
        sur = np.where(sur==True)
        k = np.max(sur)
        surr= (sur[0]/k,sur[1]/k)
        #fitt_ellipse
        el_x = None
        el_y = None
        el_maj = None
        el_min = None
        el_ang = None
        time = datetime.now()-time
        time = time.total_seconds()*1000
        data = {
            'filename': [i],
            'ellipse_center_x': [el_x],
            'ellipse_center_y': [el_y],
            'ellipse_majoraxis': [el_maj],
            'ellipse_minoraxis': [el_min],
            'ellipse_angle': [el_ang],
            'elapsed_time': [time],
        }
        save_data(pd.DataFrame(data))

In [243]:
if __name__=='__main__':
    run()

                           filename  ellipse_center_x  ellipse_center_y  \
0   2018-02-15 17.27.27.162000.tiff               NaN               NaN   
1   2018-02-15 17.47.00.307000.tiff               NaN               NaN   
2   2018-02-15 18.32.49.235000.tiff               NaN               NaN   
3   2018-05-04 13.54.29.405000.tiff               NaN               NaN   
4            2018-06-05_135014.tiff               NaN               NaN   
5   2018-06-08 14.57.44.346000.tiff               NaN               NaN   
6   2018-06-08 15.00.10.616000.tiff               NaN               NaN   
7   2018-06-08 15.07.53.070000.tiff               NaN               NaN   
8   2018-06-08 16.20.38.646000.tiff               NaN               NaN   
9            2018-07-24_111559.tiff               NaN               NaN   
10           2018-11-06_111158.tiff               NaN               NaN   

    ellipse_majoraxis  ellipse_minoraxis  ellipse_angle  elapsed_time  
0                 NaN      

In [ ]:
result = []
images = []
for i in files:
    con_img = convolve_image(contrast_image(get_image(i)))
    filter_img = filter_image(con_img)
    sur = abs((filter_img))>np.max(filter_img)*0.2
    sur = np.where(sur==True)
    print(sur)
    result.append(sur)

In [ ]:
surr= [(r[0]/np.max(r),r[1]/np.max(r)) for r in result]

In [ ]:
index = 1
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.plot(surr[index][0],surr[index][1], '.', color='black')
ax.grid(which='major', alpha=0.50)
ax.grid(which='major', color='black', linestyle='--')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(len(images), 1, figsize=(100,100))
for i,image in enumerate(images):
    axes[i].imshow(image, cmap='binary') 
    axes[i].axis('on')
plt.tight_layout()
plt.show()

    

In [ ]:
fig, axes = plt.subplots(len(convolve_img), 1, figsize=(100,100))
for i,image in enumerate(convolve_img):
    axes[i].imshow(image, cmap='binary') 
    axes[i].axis('on')
plt.tight_layout()
plt.show()